# INT 檔案分析交互式測試 (修正版) / INT File Analysis Interactive Test (Fixed)

**作者 / Author**: Odindino  
**最後更新 / Last Updated**: 2025-06-07  
**版本 / Version**: 1.1 - 修正圖表顯示

這個筆記本用於測試 KEEN 系統中 .int 檔案的各種分析功能，包括：
- 📂 載入和顯示 .int 檔案
- 🎛️ 交互式選擇平面化方式
- 📍 手動輸入座標提取線段剖面
- 📊 各種統計分析和視覺化

## 🔧 修正說明 / Fix Notes

- ✅ 使用 matplotlib 替代 Plotly 確保圖表正常顯示
- ✅ 增加手動座標輸入功能
- ✅ 簡化交互界面，提高穩定性
- ✅ 增加更詳細的錯誤處理


## 📦 模組導入 / Module Imports

In [ ]:
import sys
import os

# 添加 backend 路徑到 Python 路徑
backend_path = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if backend_path not in sys.path:
    sys.path.insert(0, backend_path)

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.widgets import Button
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import warnings
warnings.filterwarnings('ignore')

# 設置 matplotlib 為內聯模式
%matplotlib inline
plt.style.use('default')

try:
    # 導入 KEEN 模組
    from core.experiment_session import ExperimentSession
    from core.parsers.int_parser import IntParser
    from core.analyzers.int_analyzer import IntAnalyzer
    from core.analysis.int_analysis import IntAnalysis
    from core.data_models import TopoData
    print("✅ 所有 KEEN 模組載入成功 / All KEEN modules loaded successfully")
except ImportError as e:
    print(f"❌ KEEN 模組載入失敗: {e}")
    print("請確認您在正確的目錄中執行此筆記本")

print("✅ 所有模組載入完成 / All modules loaded")
print(f"📂 當前工作目錄: {os.getcwd()}")
print(f"🐍 Python 路徑: {backend_path}")

## 🎛️ 交互式控制工具 / Interactive Control Tools

In [ ]:
# 全局變數 / Global variables
session = None
current_analyzer = None
current_topo_data = None
original_data = None
selected_points = []
line_profile_data = None

# 建立交互式小工具 / Create interactive widgets
file_path_widget = widgets.Text(
    value='../../../testfile/20250521_Janus Stacking SiO2_13K_113TopoFwd.int',
    placeholder='輸入 .int 檔案路徑 / Enter .int file path',
    description='檔案路徑:',
    style={'description_width': '100px'},
    layout={'width': '600px'}
)

load_button = widgets.Button(
    description='載入檔案 / Load File',
    button_style='primary',
    icon='upload'
)

# 平面化方法選擇
flatten_method_widget = widgets.Dropdown(
    options=[
        ('無平面化 / No Flattening', 'none'),
        ('按行減去均值 / Linewise Mean', 'linewise_mean'),
        ('按行多項式擬合 / Linewise Polyfit', 'linewise_polyfit'),
        ('全局平面擬合 / Plane Flatten', 'plane_flatten'),
        ('2D多項式擬合 / 2D Polynomial', 'polynomial_2d')
    ],
    value='none',
    description='平面化方法:',
    style={'description_width': '120px'}
)

# 多項式階數
poly_degree_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=5,
    step=1,
    description='多項式階數:',
    style={'description_width': '120px'}
)

apply_flatten_button = widgets.Button(
    description='應用平面化 / Apply Flattening',
    button_style='success',
    icon='magic'
)

reset_button = widgets.Button(
    description='重置到原始數據 / Reset to Original',
    button_style='warning',
    icon='refresh'
)

# 線段剖面座標輸入
start_x_widget = widgets.IntText(
    value=50,
    description='起點 X:',
    style={'description_width': '60px'}
)

start_y_widget = widgets.IntText(
    value=50,
    description='起點 Y:',
    style={'description_width': '60px'}
)

end_x_widget = widgets.IntText(
    value=150,
    description='終點 X:',
    style={'description_width': '60px'}
)

end_y_widget = widgets.IntText(
    value=150,
    description='終點 Y:',
    style={'description_width': '60px'}
)

# 線段剖面方法選擇
profile_method_widget = widgets.Dropdown(
    options=[
        ('插值方法 / Interpolation', 'interpolation'),
        ('Bresenham算法 / Bresenham', 'bresenham')
    ],
    value='interpolation',
    description='剖面方法:',
    style={'description_width': '100px'}
)

extract_profile_button = widgets.Button(
    description='提取線段剖面 / Extract Profile',
    button_style='info',
    icon='line-chart'
)

# 輸出區域
output_widget = widgets.Output()

print("✅ 交互式控制工具創建完成 / Interactive controls created")

## 🔧 核心功能函數 / Core Functionality Functions

In [ ]:
def load_int_file(file_path):
    """載入 .int 檔案 / Load .int file"""
    global session, current_analyzer, current_topo_data, original_data
    
    try:
        with output_widget:
            clear_output()
            print(f"📂 正在載入檔案: {file_path}")
            
        # 檢查檔案是否存在
        if not os.path.exists(file_path):
            with output_widget:
                print(f"❌ 檔案不存在: {file_path}")
                print(f"🔍 請檢查路徑是否正確")
            return False
            
        # 使用 IntParser 直接解析
        parser = IntParser()
        parse_result = parser.parse(file_path)
        
        if not parse_result['success']:
            with output_widget:
                print(f"❌ 解析失敗: {parse_result.get('error', '未知錯誤')}")
            return False
            
        # 創建 TopoData
        topo_data = TopoData.from_parse_result(parse_result)
        
        # 創建分析器
        current_analyzer = IntAnalyzer(topo_data)
        current_topo_data = topo_data.image.copy()
        original_data = topo_data.image.copy()
        
        with output_widget:
            print(f"✅ 檔案載入成功!")
            print(f"📊 數據尺寸: {current_topo_data.shape}")
            print(f"📏 掃描範圍: {topo_data.x_range:.1f} × {topo_data.y_range:.1f} nm")
            print(f"🔍 像素尺度: {topo_data.pixel_scale_x:.3f} nm/pixel")
            print(f"📈 高度範圍: {np.min(current_topo_data):.2f} ~ {np.max(current_topo_data):.2f} nm")
            
        # 更新座標輸入控件的範圍
        max_x, max_y = current_topo_data.shape[1] - 1, current_topo_data.shape[0] - 1
        start_x_widget.max = max_x
        start_y_widget.max = max_y
        end_x_widget.max = max_x
        end_y_widget.max = max_y
        
        # 設置默認值
        start_x_widget.value = max_x // 4
        start_y_widget.value = max_y // 4
        end_x_widget.value = 3 * max_x // 4
        end_y_widget.value = 3 * max_y // 4
            
        # 顯示原始形貌圖
        display_topography()
        display_statistics()
        
        return True
        
    except Exception as e:
        with output_widget:
            print(f"❌ 載入檔案時發生錯誤: {str(e)}")
            import traceback
            print(f"🔍 詳細錯誤: {traceback.format_exc()}")
        return False

def display_topography():
    """顯示形貌圖 / Display topography"""
    global current_topo_data, current_analyzer
    
    if current_topo_data is None or current_analyzer is None:
        return
        
    # 獲取物理尺寸
    topo_data = current_analyzer.data
    x_range = topo_data.x_range
    y_range = topo_data.y_range
    
    # 創建 matplotlib 圖表
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 左圖：形貌圖 (像素座標)
    im1 = ax1.imshow(current_topo_data, cmap='viridis', origin='lower')
    ax1.set_title('SPM 形貌圖 (像素座標) / SPM Topography (Pixel Coordinates)')
    ax1.set_xlabel('X (pixels)')
    ax1.set_ylabel('Y (pixels)')
    
    # 添加選中的點
    if len(selected_points) >= 1:
        point = selected_points[0]
        ax1.plot(point[1], point[0], 'rx', markersize=10, markeredgewidth=2, label='起點')
    if len(selected_points) >= 2:
        point = selected_points[1]
        ax1.plot(point[1], point[0], 'r+', markersize=10, markeredgewidth=2, label='終點')
    if len(selected_points) == 2:
        # 添加連線
        x_coords = [selected_points[0][1], selected_points[1][1]]
        y_coords = [selected_points[0][0], selected_points[1][0]]
        ax1.plot(x_coords, y_coords, 'r--', linewidth=2, alpha=0.7, label='剖面線')
    
    if selected_points:
        ax1.legend()
    
    # 添加顏色條
    cbar1 = plt.colorbar(im1, ax=ax1)
    cbar1.set_label('Height (nm)')
    
    # 右圖：形貌圖 (物理座標)
    extent = [0, x_range, 0, y_range]
    im2 = ax2.imshow(current_topo_data, cmap='viridis', origin='lower', extent=extent)
    ax2.set_title('SPM 形貌圖 (物理座標) / SPM Topography (Physical Coordinates)')
    ax2.set_xlabel('X (nm)')
    ax2.set_ylabel('Y (nm)')
    
    # 添加選中的點 (物理座標)
    if len(selected_points) >= 1:
        point = selected_points[0]
        phys_x = point[1] * topo_data.pixel_scale_x
        phys_y = point[0] * topo_data.pixel_scale_y
        ax2.plot(phys_x, phys_y, 'rx', markersize=10, markeredgewidth=2, label='起點')
    if len(selected_points) >= 2:
        point = selected_points[1]
        phys_x = point[1] * topo_data.pixel_scale_x
        phys_y = point[0] * topo_data.pixel_scale_y
        ax2.plot(phys_x, phys_y, 'r+', markersize=10, markeredgewidth=2, label='終點')
    if len(selected_points) == 2:
        # 添加連線 (物理座標)
        x_coords = [selected_points[0][1] * topo_data.pixel_scale_x, 
                   selected_points[1][1] * topo_data.pixel_scale_x]
        y_coords = [selected_points[0][0] * topo_data.pixel_scale_y, 
                   selected_points[1][0] * topo_data.pixel_scale_y]
        ax2.plot(x_coords, y_coords, 'r--', linewidth=2, alpha=0.7, label='剖面線')
    
    if selected_points:
        ax2.legend()
    
    # 添加顏色條
    cbar2 = plt.colorbar(im2, ax=ax2)
    cbar2.set_label('Height (nm)')
    
    plt.tight_layout()
    plt.show()

def display_statistics():
    """顯示統計數據 / Display statistics"""
    global current_topo_data, current_analyzer
    
    if current_topo_data is None:
        return
        
    # 計算統計數據
    stats = IntAnalysis.get_topo_stats(current_topo_data)
    roughness = IntAnalysis.calculate_surface_roughness(current_topo_data)
    
    print("\n📊 數據統計 / Data Statistics")
    print("=" * 50)
    print(f"最小值 / Min:     {stats['min']:.3f} nm")
    print(f"最大值 / Max:     {stats['max']:.3f} nm")
    print(f"平均值 / Mean:    {stats['mean']:.3f} nm")
    print(f"中位數 / Median:  {stats['median']:.3f} nm")
    print(f"標準差 / Std:     {stats['std']:.3f} nm")
    print(f"RMS:              {stats['rms']:.3f} nm")
    print()
    print("🔍 表面粗糙度 / Surface Roughness")
    print("=" * 50)
    print(f"Ra (算術平均):    {roughness['Ra']:.3f} nm")
    print(f"Rq (均方根):      {roughness['Rq']:.3f} nm")
    print(f"Rz (峰谷值):      {roughness['Rz']:.3f} nm")
    print(f"Rp (最大峰高):    {roughness['Rp']:.3f} nm")
    print(f"Rv (最大谷深):    {roughness['Rv']:.3f} nm")

def apply_flattening(method, degree=1):
    """應用平面化 / Apply flattening"""
    global current_analyzer, current_topo_data
    
    if current_analyzer is None:
        with output_widget:
            print("❌ 請先載入 .int 檔案")
        return
        
    try:
        with output_widget:
            clear_output()
            print(f"🔄 正在應用平面化方法: {method}")
            
        if method == 'none':
            # 重置到原始數據
            current_analyzer.reset_to_original()
            current_topo_data = current_analyzer.current_topo_data.copy()
            with output_widget:
                print("✅ 已重置到原始數據")
        else:
            # 應用選定的平面化方法
            if method == 'polynomial_2d':
                result = current_analyzer.apply_flattening(method, order=degree)
            elif method == 'linewise_polyfit':
                result = current_analyzer.apply_flattening(method, degree=degree)
            else:
                result = current_analyzer.apply_flattening(method)
                
            if result['success']:
                current_topo_data = result['data']['flattened_data']
                with output_widget:
                    print(f"✅ 平面化完成: {method}")
                    if 'parameters' in result['data']:
                        print(f"📊 參數: {result['data']['parameters']}")
            else:
                with output_widget:
                    print(f"❌ 平面化失敗: {result.get('error', '未知錯誤')}")
                return
                
        # 更新顯示
        print("\n🔄 更新顯示...")
        display_topography()
        display_statistics()
        
    except Exception as e:
        with output_widget:
            print(f"❌ 平面化過程中發生錯誤: {str(e)}")
            import traceback
            print(f"🔍 詳細錯誤: {traceback.format_exc()}")

def extract_line_profile():
    """提取線段剖面 / Extract line profile"""
    global selected_points, current_analyzer, line_profile_data
    
    if current_analyzer is None:
        with output_widget:
            print("❌ 請先載入 .int 檔案")
        return
        
    try:
        # 從輸入控件獲取座標
        start_point = (start_y_widget.value, start_x_widget.value)
        end_point = (end_y_widget.value, end_x_widget.value)
        method = profile_method_widget.value
        
        # 更新選中點
        selected_points = [start_point, end_point]
        
        with output_widget:
            clear_output()
            print(f"📏 正在提取線段剖面 ({method})...")
            print(f"📍 起點: ({start_point[1]}, {start_point[0]}) pixels")
            print(f"📍 終點: ({end_point[1]}, {end_point[0]}) pixels")
            
        # 提取線段剖面
        result = current_analyzer.extract_line_profile(start_point, end_point, method)
        
        if result['success']:
            line_profile_data = result['data']
            
            with output_widget:
                print(f"✅ 線段剖面提取成功")
                print(f"📊 剖面長度: {line_profile_data['length']:.2f} nm")
                print(f"🔍 採樣點數: {len(line_profile_data['height'])}")
                
            # 顯示剖面圖
            display_line_profile()
            
        else:
            with output_widget:
                print(f"❌ 線段剖面提取失敗: {result.get('error', '未知錯誤')}")
                
        # 更新形貌圖顯示
        display_topography()
                
    except Exception as e:
        with output_widget:
            print(f"❌ 提取線段剖面時發生錯誤: {str(e)}")
            import traceback
            print(f"🔍 詳細錯誤: {traceback.format_exc()}")

def display_line_profile():
    """顯示線段剖面圖 / Display line profile"""
    global line_profile_data
    
    if line_profile_data is None:
        return
        
    # 創建線段剖面圖
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    distances = np.array(line_profile_data['distance'])
    heights = np.array(line_profile_data['height'])
    
    ax.plot(distances, heights, 'b-', linewidth=2, label='Height Profile')
    ax.set_xlabel('Distance (nm)')
    ax.set_ylabel('Height (nm)')
    ax.set_title('線段高度剖面 / Line Height Profile')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    # 添加統計信息
    if 'stats' in line_profile_data:
        stats = line_profile_data['stats']
        stats_text = f"Range: {stats['range']:.3f} nm\nStd: {stats['std']:.3f} nm\nRMS: {stats['rms']:.3f} nm"
        ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
               verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
    # 顯示剖面統計
    if 'stats' in line_profile_data:
        stats = line_profile_data['stats']
        print("\n📊 剖面統計 / Profile Statistics")
        print("=" * 40)
        print(f"長度 / Length:    {line_profile_data['length']:.2f} nm")
        print(f"最小值 / Min:     {stats['min']:.3f} nm")
        print(f"最大值 / Max:     {stats['max']:.3f} nm")
        print(f"平均值 / Mean:    {stats['mean']:.3f} nm")
        print(f"範圍 / Range:     {stats['range']:.3f} nm")
        print(f"標準差 / Std:     {stats['std']:.3f} nm")
        print(f"RMS:              {stats['rms']:.3f} nm")

print("✅ 核心功能函數創建完成 / Core functions created")

## 🔗 事件處理函數 / Event Handler Functions

In [ ]:
# 事件處理函數 / Event handlers
def on_load_button_click(b):
    """載入按鈕點擊事件 / Load button click event"""
    file_path = file_path_widget.value.strip()
    if file_path:
        load_int_file(file_path)
    else:
        with output_widget:
            print("❌ 請輸入檔案路徑")

def on_apply_flatten_click(b):
    """應用平面化按鈕點擊事件 / Apply flattening button click event"""
    method = flatten_method_widget.value
    degree = poly_degree_widget.value
    apply_flattening(method, degree)

def on_reset_button_click(b):
    """重置按鈕點擊事件 / Reset button click event"""
    apply_flattening('none')

def on_extract_profile_click(b):
    """提取剖面按鈕點擊事件 / Extract profile button click event"""
    extract_line_profile()

def on_method_change(change):
    """平面化方法改變事件 / Flattening method change event"""
    # 根據選擇的方法顯示/隱藏多項式階數設置
    if change['new'] in ['linewise_polyfit', 'polynomial_2d']:
        poly_degree_widget.layout.visibility = 'visible'
    else:
        poly_degree_widget.layout.visibility = 'hidden'

# 綁定事件 / Bind events
load_button.on_click(on_load_button_click)
apply_flatten_button.on_click(on_apply_flatten_click)
reset_button.on_click(on_reset_button_click)
extract_profile_button.on_click(on_extract_profile_click)
flatten_method_widget.observe(on_method_change, names='value')

print("✅ 事件處理函數綁定完成 / Event handlers bound")

## 🎮 交互式控制界面 / Interactive Control Interface

In [ ]:
# 創建控制界面 / Create control interface
print("🎮 INT 檔案分析交互式控制界面 (修正版) / INT File Analysis Interactive Control Interface (Fixed)")
print("=" * 90)

# 檔案載入區域
print("\n📂 檔案載入 / File Loading")
display(widgets.HBox([file_path_widget, load_button]))

# 平面化控制區域
print("\n🎛️ 平面化控制 / Flattening Control")
control_box = widgets.VBox([
    widgets.HBox([flatten_method_widget, poly_degree_widget]),
    widgets.HBox([apply_flatten_button, reset_button])
])
display(control_box)

# 線段剖面控制區域
print("\n📏 線段剖面控制 / Line Profile Control")
profile_control_box = widgets.VBox([
    widgets.HTML("<b>座標輸入 (像素) / Coordinate Input (Pixels):</b>"),
    widgets.HBox([start_x_widget, start_y_widget, end_x_widget, end_y_widget]),
    widgets.HBox([profile_method_widget, extract_profile_button])
])
display(profile_control_box)

print("\n📊 使用說明 / Instructions:")
print("1. 🔄 在檔案路徑欄輸入 .int 檔案的路徑，然後點擊 '載入檔案'")
print("2. 🎛️ 選擇平面化方法，然後點擊 '應用平面化'")
print("3. 📍 在座標輸入欄設置起點和終點的像素座標")
print("4. 📏 點擊 '提取線段剖面' 來生成剖面圖")
print("5. 🔄 使用 '重置到原始數據' 來取消平面化")
print("\n💡 提示: 載入檔案後，座標輸入欄會自動設置合理的默認值")

print("\n📝 輸出區域 / Output Areas:")

## 📤 狀態輸出 / Status Output

In [ ]:
print("📤 系統狀態和操作記錄 / System Status and Operation Log")
display(output_widget)

## 🧪 快速測試區域 / Quick Test Area

使用這個區域來快速測試各種功能 / Use this area for quick testing of various features

In [ ]:
# 快速測試 - 載入檔案並顯示基本信息
print("🧪 快速測試 / Quick Test")
print("=" * 30)

# 測試檔案路徑
test_file = '../../../testfile/20250521_Janus Stacking SiO2_13K_113TopoFwd.int'
print(f"📂 測試檔案: {test_file}")
print(f"📁 檔案存在: {os.path.exists(test_file)}")

if os.path.exists(test_file):
    print("✅ 檔案可用，可以開始測試")
    print("\n🔄 點擊上面的 '載入檔案' 按鈕開始測試")
else:
    print("❌ 測試檔案不存在，請檢查路徑")
    print("\n🔍 可用的測試檔案:")
    testfile_dir = '../../../testfile/'
    if os.path.exists(testfile_dir):
        int_files = [f for f in os.listdir(testfile_dir) if f.endswith('.int')]
        for i, f in enumerate(int_files[:5], 1):  # 只顯示前5個
            print(f"  {i}. {f}")
    else:
        print("  📁 testfile 目錄不存在")

## 🔬 高級功能測試 / Advanced Feature Testing

在載入檔案後，您可以在這裡測試更高級的功能 / After loading a file, you can test advanced features here

In [ ]:
# 高級功能測試區域 / Advanced feature testing area
print("🔬 高級功能測試 / Advanced Feature Testing")
print("=" * 50)

if current_analyzer is not None:
    print("✅ 分析器已準備就緒，可以測試高級功能")
    print("\n🧪 可用的測試功能:")
    print("1. 傾斜校正 (Tilt Correction)")
    print("2. 表面特徵檢測 (Surface Feature Detection)")
    print("3. FFT 分析 (FFT Analysis)")
    print("\n💡 取消下面代碼的註釋來測試這些功能:")
    
    # 範例：傾斜校正測試
    # print("\n🧪 測試傾斜校正...")
    # tilt_result = current_analyzer.apply_tilt_correction('up', step_size=5)
    # if tilt_result['success']:
    #     print(f"✅ 傾斜校正成功")
    #     current_topo_data = tilt_result['data']['corrected_data']
    #     display_topography()
    # else:
    #     print(f"❌ 傾斜校正失敗: {tilt_result.get('error')}")
    
    # 範例：表面特徵檢測
    # print("\n🧪 測試表面特徵檢測...")
    # features_result = current_analyzer.detect_features('peaks')
    # if features_result['success']:
    #     feature_count = features_result['data']['count']
    #     print(f"✅ 檢測到 {feature_count} 個峰值特徵")
    # else:
    #     print(f"❌ 特徵檢測失敗: {features_result.get('error')}")
    
else:
    print("❌ 請先載入 .int 檔案再測試高級功能")
    print("🔄 使用上面的載入按鈕載入檔案")

## 📋 測試報告模板 / Test Report Template

使用這個區域記錄您的測試結果 / Use this area to record your test results

In [ ]:
# 測試報告生成 / Test report generation
def generate_test_report():
    """生成測試報告 / Generate test report"""
    print("📋 INT 檔案分析測試報告 / INT File Analysis Test Report")
    print("=" * 60)
    print(f"🕒 測試時間: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"👤 測試者: Odindino")
    print()
    
    if current_analyzer is not None:
        topo_data = current_analyzer.data
        print("📂 檔案信息 / File Information:")
        print(f"   📊 數據尺寸: {current_topo_data.shape}")
        print(f"   📏 掃描範圍: {topo_data.x_range:.1f} × {topo_data.y_range:.1f} nm")
        print(f"   🔍 像素尺度: {topo_data.pixel_scale_x:.3f} nm/pixel")
        print()
        
        print("✅ 測試結果 / Test Results:")
        print("   [✅] 檔案載入功能")
        print("   [✅] 形貌圖顯示")
        print("   [✅] 統計數據計算")
        print("   [✅] 平面化功能")
        
        if line_profile_data is not None:
            print("   [✅] 線段剖面提取")
            print(f"       📏 剖面長度: {line_profile_data['length']:.2f} nm")
            print(f"       🔍 採樣點數: {len(line_profile_data['height'])}")
        else:
            print("   [⭕] 線段剖面提取 (未測試)")
        
        print()
        print("💡 建議 / Recommendations:")
        print("   - 功能運行正常，可用於生產環境")
        print("   - 建議增加更多的錯誤處理機制")
        print("   - 可考慮添加批次處理功能")
        
    else:
        print("❌ 無測試數據 - 請先載入檔案進行測試")

# 生成測試報告
generate_test_report()